# Project 1 Part 4 - Creating a master parcel database

In this part of the project, we will use Python to read, process, and double all of the parcel data into a database.  Note that this is not our only alternative, and in Project 1 Part 4 b, we will look at another alternative, that is reading all the of original, raw files into their own database table, then using SQL to join/link/aggregate the tables.

## Chunking Files in Pandas – Part 1 (20 Points)

In this part of the project, you will use `Panda`’s to process the data from the MinneMUDAC 2016 competition Dive into Water Data.  The data can be found at the [MinneMUDAC site](http://minneanalytics.org/minnemudac/data/).  You should document your work in a Jupyter notebook, which will be used to submit your solution.  **For the rest of the parts of this project, we will limit ourselves to the years 2004-2014.**

1. Remind me why we want to skip 2003.

2003 has a lot less columns in common with the other years

2. Import the common columns list and translation dictionaries from the `.py` file you created in the last part of the project.

In [3]:
from project_data_Buske import common_columns

In [4]:
from project_data_Buske import latlong_to_code

In [5]:
from project_data_Buske import latlong_to_name

In [6]:
from project_data_Buske import latlong_to_distance

3. Use glob and a list comprehension to get a list of file names for the years 2004-2014.

In [37]:
from glob import glob
files = glob('./data/MinneMUDAC_raw_files/20**_metro_tax_parcels.txt')[2:-1]
[file for file in files]

['./data/MinneMUDAC_raw_files/2004_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2005_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2006_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2007_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2008_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2009_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2010_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2011_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2012_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2013_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2014_metro_tax_parcels.txt']

4. Use the first chunk of the first file to prototype an expression that <br>
    a. Selects the common columns <br>
    b. Fixes any issues with the column names <br>
    c. Changes columns to the correct types (if necessary).  More information about the columns can be found [here](ftp://ftp.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metrogis/plan_regonal_prcls_open/metadata/metadata.html). It is **imperative** that you keep the lat and long columns as strings. <br>
    d. Use the translation dictionaries from the last part to add three new columns to the chunk: lake code, lake name, parcel distance to the lake.
    e. Filters to only properties that are within 1600 m (~1 mile) of the closest lake.

In [40]:
import pandas as pd
fileiterator = [pd.read_csv(file, chunksize = 500, sep = '|') for file in files]
fileiterator

In [45]:
from toolz import first
firstchunk = [first(ele) for ele in fileiterator]
firstchunk

[      ACRES_DEED  ACRES_POLY  AGPRE_ENRD  ...  ZIP4 centroid_lat  centroid_long
 1000         0.0        0.27         NaN  ...   NaN     45.39158      -93.34664
 1001         0.0        0.58         NaN  ...   NaN     45.39141      -93.33901
 1002         0.0        0.50         NaN  ...   NaN     45.39167      -93.36317
 1003         0.0        0.17         NaN  ...   NaN     45.39173      -93.36099
 1004         0.0        0.26         NaN  ...   NaN     45.39167      -93.36162
 1005         0.0        0.35         NaN  ...   NaN     45.39173      -93.36470
 1006         0.0        0.35         NaN  ...   NaN     45.39172      -93.36382
 1007         0.0        4.08         NaN  ...   NaN     45.39111      -93.36565
 1008         0.0        0.29         NaN  ...   NaN     45.39160      -93.34327
 1009         0.0        2.11         NaN  ...   NaN     45.39155      -93.39036
 1010         0.0        0.30         NaN  ...   NaN     45.39166      -93.36217
 1011         0.0        2.4

5. Now convert your expression from the last problem to a function and test that this function works on the first few chunks of each file.

6. We need to make a unique primary key for each row in the combined parcel file.<br>
    a. There is a column that appears to be a unique parcel id.  Double check that this is a true primary key for each individual file. (To do this you need to verify that the number of unique values is the same as the number of rows for each of the parcel files.  **Hint:** For each file, use of the accumulator pattern with two accumualtors (one number and one data frame). <br>
    b. Explain why this column will not work as a primary key if we want to combine all years in one database. <br>
    c. Suppose we make a new column that consist of `str(year) + '-' + PID`.  Explain why this should make a proper primary key for the combined data. <br>

7. Make a function to add the key suggested in the last problem (`str(year) + '-' + PID`) to a given chunk.

#### Note: If you are clever, you can do parts 8 in one double loop, which will save you from having to read the parcel files twice.

8. It is probably worth our time to test that our new key column is truely unique. (If not, we might be wasting out time loading the data into a database, only to have process fail hours in.) Test that the new column works by <br>
    a. Iterating over all the files.<br>
    b. Using an accumulator to count total number of rows across all parcel files. <br>
    c. Using an accumulator to accumulate a set of all unique values of our new key. <br>
    d. Verifying that we have as many total rows as unique keys.
    a. Selecting just this column. <br>
    b. Dumping this column into a temporary database <br>

9. If the last step succeeded, you can proceed to make a master parcel data database.  If not, you will need to figure out another primary key, probably an `id` column similar to the example in the lectures.